In [1]:
model_name = 'xlm-roberta-finetuned-financial-news-sentiment-analysis-european'

In [2]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nestorojedagonzalez (nojeda) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
from datasets import load_dataset
ds = load_dataset("nojedag/financial_phrasebank_multilingual")

README.md:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/734k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16357 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7011 [00:00<?, ? examples/s]

In [4]:
# Define the transform_labels function to handle batched input
def transform_labels(examples):
	label_map = {"negative": 0, "neutral": 1, "positive": 2}
	if isinstance(examples['sentiment'], list):
		examples['labels'] = [label_map[s.lower()] for s in examples['sentiment']]
	else:
		examples['labels'] = label_map[examples['sentiment'].lower()]
	return examples

dataset = ds.map(transform_labels, batched=True)

Map:   0%|          | 0/16357 [00:00<?, ? examples/s]

Map:   0%|          | 0/7011 [00:00<?, ? examples/s]

In [5]:
# let's train a Distilbert model

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base', num_labels=3)

# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['sentence'], padding='max_length', truncation=True)    

In [6]:
dataset = dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/16357 [00:00<?, ? examples/s]

Map:   0%|          | 0/7011 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base", num_labels=3)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# the default batch size for training arguments
batch_size = 32

# set number of epochs
number_of_epochs = 4
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.1 * steps)

In [9]:
import torch
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, weight_tensor, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = torch.nn.CrossEntropyLoss(weight=weight_tensor.to(self.model.device))

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = self.loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [10]:
import numpy as np
import evaluate

metric=evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    return metric.compute(predictions=predictions, references=labels)

In [11]:
from transformers import TrainingArguments
from utils import get_output_dir
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  eval_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  weight_decay=0.01,  # added weight decay
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir=get_output_dir(model_name),
                                  report_to="wandb",
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  gradient_accumulation_steps=2,  # accumulate gradients over 2 steps
                                  fp16=True
                                )

In [12]:
train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['test'].shuffle(seed=10)

In [13]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
from transformers import Trainer
weight_tensor = torch.tensor([1.0, 2.0, 3.0])

trainer = CustomTrainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, weight_tensor=weight_tensor, data_collator=data_collator, compute_metrics=compute_metrics
)

In [15]:
# Launch the learning process: training
# Load model from checkpoint
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy
500,No log,0.337552,0.808159
1000,0.574700,0.299559,0.834831


TrainOutput(global_step=1024, training_loss=0.4031321526854299, metrics={'train_runtime': 1368.4659, 'train_samples_per_second': 47.811, 'train_steps_per_second': 0.748, 'total_flos': 1.7214984695107584e+16, 'train_loss': 0.4031321526854299, 'epoch': 4.0})

In [16]:

trainer_eval = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [17]:
trainer_eval.evaluate()

{'eval_loss': 0.38536590337753296,
 'eval_model_preparation_time': 0.0045,
 'eval_accuracy': 0.8348309798887462,
 'eval_runtime': 21.6518,
 'eval_samples_per_second': 323.807,
 'eval_steps_per_second': 10.161}

In [18]:
model.save_pretrained(model_name)
model.push_to_hub(f'nojedag/{model_name}')

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european/commit/f0ee75942592fc1343a560c5b56fd50c955addb9', commit_message='Upload XLMRobertaForSequenceClassification', commit_description='', oid='f0ee75942592fc1343a560c5b56fd50c955addb9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european'), pr_revision=None, pr_num=None)

In [19]:
trainer.push_to_hub()
trainer_eval.push_to_hub()

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european/commit/e9b26a52a9d7352f54e8b27119e151b7c0a80c31', commit_message='End of training', commit_description='', oid='e9b26a52a9d7352f54e8b27119e151b7c0a80c31', pr_url=None, repo_url=RepoUrl('https://huggingface.co/nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european', endpoint='https://huggingface.co', repo_type='model', repo_id='nojedag/xlm-roberta-finetuned-financial-news-sentiment-analysis-european'), pr_revision=None, pr_num=None)

In [20]:
wandb.finish()

eval/accuracy,▁██
eval/loss,▄▁█
eval/model_preparation_time,▁
eval/runtime,▂█▁
eval/samples_per_second,▇▁█
eval/steps_per_second,▇▁█
train/epoch,▁▁███
train/global_step,▄▄███▁
train/grad_norm,▁█
train/learning_rate,█▁
train/loss,█▁
